## 建立檢索增強生成 (RAG) 應用程式
- https://python.langchain.com/docs/tutorials/rag/
- https://medium.com/jimmy-wang/langchain-rag%E5%AF%A6%E6%88%B0%E7%AC%AC%E4%B8%80%E7%AB%99-efe975f4c3bd

In [2]:
# 安裝套件
!uv pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph langchain langchain-openai langchain-nvidia-ai-endpoints langchain-chroma

In [42]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [4]:
# LLM 模型
# https://build.nvidia.com/deepseek-ai/deepseek-r1
# nvapi-
import getpass
import os

if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("meta/llama-4-maverick-17b-128e-instruct", model_provider="nvidia")

Enter API key for NVIDIA:  ········


D:\uv\langchaing\Lib\site-packages\langchain_nvidia_ai_endpoints\_common.py:212: UserWarning: Found meta/llama-4-maverick-17b-128e-instruct in available_models, but type is unknown and inference may fail.
  warnings.warn(


In [5]:
# embedding and rerank 模型
# https://jina.ai/
# jina_
import getpass
import os

if not os.environ.get("JINA_API_KEY"):
  os.environ["JINA_API_KEY"] = getpass.getpass("Enter API key for Voyage AI: ")

from langchain_community.embeddings import JinaEmbeddings
embeddings = JinaEmbeddings(
    jina_api_key=os.environ["JINA_API_KEY"], model_name="jina-embeddings-v3"
)


from langchain_community.document_compressors import JinaRerank

rerank = JinaRerank(model="jina-reranker-v2-base-multilingual")  # 或使用 "rerank-1" 精準但較慢

Enter API key for Voyage AI:  ········


In [7]:
# 向量資料庫建立
from langchain_chroma import Chroma

vector_store = Chroma(
    #collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db27",  # Where to save data locally, remove if not necessary
)

### Indexing
我們需要先載入部落格文章內容。我們可以為此使用 DocumentLoaders ，它是從來源載入資料並傳回 Document物件 清單的物件。
在這種情況下，我們將使用 WebBaseLoader，它用於urllib從 Web URL 載入 HTML 並將BeautifulSoup其解析為文字。我們可以透過向BeautifulSoup解析器傳遞參數來客製化 HTML -> 文字解析bs_kwargs（參見 BeautifulSoup 文件）。在這種情況下，只有帶有“post-content”，“post-title”或“post-header”類別的 HTML 標籤是相關的，因此我們將刪除所有其他標籤。

In [8]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
# <div id="mw-content-text" class="mw-body-content">
#bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
bs4_strainer = bs4.SoupStrainer(class_=("mw-body-content"))
loader = WebBaseLoader(
    #web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    web_paths=("https://zh.wikipedia.org/zh-tw/%E9%B9%BF%E9%BC%8E%E8%AE%B0",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

#assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")
print(docs[0].page_content[:500])

USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 18282
關於與「鹿鼎記」標題相近或相同的條目，請見「鹿鼎記 (消歧義)」。
鹿鼎記書籍封面作者金庸類型小說語言中文故事背景主題武俠時代清朝 出版資訊出版機構 明河社 廣州出版社 遠流出版社出版日期1969年出版地 英屬香港媒介文字；報紙連載；書冊系列作品前作笑傲江湖續作越女劍
《鹿鼎記》是香港作家金庸的最後一部武俠小說作品，可以視為金庸創作的最高峰、最頂點。1969年10月24日至1972年9月23日連載於《明報》，共22回。小說故事發生在清初（1668年-1689年），以寫實主義、歷史引述和悲劇主題喜劇化的手法見稱，與《碧血劍》的情節有所關連。


故事梗概[編輯]
康熙初年，浙江富紳莊允誠遭革職知縣吳之榮舉報，告發其子所編《明史》中使用前明年號。權臣鰲拜即大興文字獄，莊家遭全家抄斬，更牽連有份編修，以至買賣此籍之民眾，這便是轟動全國的莊廷鑨明史案。
小說的主人公韋小寶成長於揚州妓院麗春院，母親韋春花是院內妓女。十二三歲的韋小寶救了越獄犯茅十八一命，茅十八誇口要上京與「滿洲第一勇士」鰲拜決鬥，韋小寶便隨他北上。
兩人在京城遭尚膳監總管海大富擒進宮中。韋小寶暗中將海大富毒盲，把服侍他的小太監小


### Splitting documents
我們載入的文檔超過 18k 個字符，太長，無法放入許多模型的上下文視窗。即使對於那些可以在其上下文視窗中容納完整帖子的模型，模型也很難在很長的輸入中找到資訊。

為了解決這個問題，我們將其分成Document用於嵌入和向量儲存的區塊。這應該有助於我們在運行時僅檢索部落格文章中最相關的部分。

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=100,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 24 sub-documents.


### Storing documents

現在我們需要索引我們的 24 個文字區塊，以便我們可以在運行時搜尋它們。按照語義搜尋教程，我們的方法是嵌入每個文件分割的內容並將這些嵌入插入到向量儲存中。給定輸入查詢，我們可以使用向量搜尋來檢索相關文件。

In [10]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['2c43cc40-3f92-4449-8ff4-b8166fc7ba1f', 'a6f50243-f803-436a-a98b-879d538ceac4', 'af78033b-3f49-4a51-8196-b55042620392']


In [11]:
for i in range(min(3, len(all_splits))):
    print(f"--- Document {i+1} ---")
    print(all_splits[i].page_content[:500]) # Printing the first 500 characters
    print("\n")

--- Document 1 ---
關於與「鹿鼎記」標題相近或相同的條目，請見「鹿鼎記 (消歧義)」。
鹿鼎記書籍封面作者金庸類型小說語言中文故事背景主題武俠時代清朝 出版資訊出版機構 明河社 廣州出版社 遠流出版社出版日期1969年出版地 英屬香港媒介文字；報紙連載；書冊系列作品前作笑傲江湖續作越女劍
《鹿鼎記》是香港作家金庸的最後一部武俠小說作品，可以視為金庸創作的最高峰、最頂點。1969年10月24日至1972年9月23日連載於《明報》，共22回。小說故事發生在清初（1668年-1689年），以寫實主義、歷史引述和悲劇主題喜劇化的手法見稱，與《碧血劍》的情節有所關連。


--- Document 2 ---
故事梗概[編輯]
康熙初年，浙江富紳莊允誠遭革職知縣吳之榮舉報，告發其子所編《明史》中使用前明年號。權臣鰲拜即大興文字獄，莊家遭全家抄斬，更牽連有份編修，以至買賣此籍之民眾，這便是轟動全國的莊廷鑨明史案。
小說的主人公韋小寶成長於揚州妓院麗春院，母親韋春花是院內妓女。十二三歲的韋小寶救了越獄犯茅十八一命，茅十八誇口要上京與「滿洲第一勇士」鰲拜決鬥，韋小寶便隨他北上。
兩人在京城遭尚膳監總管海大富擒進宮中。韋小寶暗中將海大富毒盲，把服侍他的小太監小桂子殺死後，冒充為小桂子以應付眼盲的海大富，助茅十八脫身。韋小寶受海大富命令，在宮中找尋《四十二章經》，無意中結識了自稱「小玄子」的康熙皇帝及建寧公主，成為好友，更助康熙生擒鰲拜。海大富憑著小寶與康熙比武的招式，推敲出殺死康熙生母孝康章皇后，以及孝獻端敬皇后及其子榮親王等人的兇手正是太后，依照順治帝從前的命令，往取太后性命，激鬥後卻為太后所殺。而韋小寶從海大富與太后之間的對話中，得知康熙帝之父順治帝尚在人間。
韋小寶奉命去殺囚在康親王府中的鰲拜，竟遇上欲刺殺鰲拜的天地會一眾，韋小寶在混亂中殺死了鰲拜，之後被一眾天地會刺客擄走。天地會總舵主陳近南


--- Document 3 ---
康熙得知順治在清涼寺，為掩人耳目，先派韋小寶到少林寺出家。路上，韋小寶及眾侍衛遭王屋派偷襲，由此認識了少女曾柔，韋小寶藉寶刀寶衣反制王屋派，卻又借賭局將他們釋放。
韋小寶剃度後，在寺外遇到來少林寺挑釁的阿珂，深深為其美貌傾倒。韋小寶後來轉往清涼寺當住持，大批喇嘛再次圍攻清涼寺，小寶施計帶領眾人逃出，剛好遇上康熙率眾前來。眾喇嘛被

In [38]:
# Step 2: 把向量資料庫轉換為 retriever，並指定檢索參數
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
query = "誰是蘇菲亞公主"
docs = retriever.invoke(query)
print(docs)
#pretty_print_docs(docs)

[Document(id='77220522-d437-4f9e-b941-246640fcb5ed', metadata={'source': 'https://zh.wikipedia.org/zh-tw/%E9%B9%BF%E9%BC%8E%E8%AE%B0', 'start_index': 11828}, page_content='蘇荃\n馬海倫\n黃愷欣\n林秀君\n馮曉文\n陳法蓉（龍兒）\n胡可\n張馨予\n朱珠\n\n\n建寧公主\n梁翠紅\n景黛音\n鄭學琳\n劉玉翠\n林心如\n舒暢\n婁藝瀟\n唐藝昕\n\n\n曾柔\n李海敏\n吳君如\n-\n陳安琪\n舒淇（小金魚）\n李菲兒\n王雅慧\n鍾麗麗\n\n\n阿珂\n花居冠\n商天娥\n陳玉玫\n梁小冰\n朱茵\n應采兒\n賈青\n郭泱\n\n\n海大富\n-\n劉兆銘\n陳慧樓\n谷峰\n吳孟達\n華子\n計春華\n田雨\n\n\n皇太后\n-\n呂有慧\n胡錦\n程可為\n陳法蓉\n高遠\n米雪\n王秀竹\n\n\n鰲拜\n-\n關海山\n林照雄\n王俊棠\n徐錦江\n賴水清\n劉天佐\n\n\n多隆\n-\n許紹雄\n-\n艾威\n黃一飛\n胡東\n劉永\n劉超\n\n\n九難\n-\n陳嘉儀\n於文惠\n李麗麗\n李菲\n何佳怡\n王婉娟\n李淑桐\n\n\n鄭克塽\n-\n戴志偉\n談學斌\n郭耀明\n鄭瑞曉\n喬振宇\n林江國\n曾柯琅\n\n\n馮錫范\n-\n葉天行\n王圻生薛漢\n郭德信\n張春仲\n于承惠\n孟飛\n王宏偉\n\n\n吳三桂\n-\n劉丹\n黃永光\n王偉\n張振寰\n圖門\n馮進高\n于榮光\n\n\n吳應熊\n-\n張志強\n宋憲宏\n戴志偉\n卓凡\n鍾亮\n古斌\n郭乙桓\n\n\n洪安通\n曹達華\n陳有后\n關洪\n鮑方\n陳觀泰\n袁苑\n盧勇\n王德順\n\n\n蘇菲亞公主\n-\n狄寶娜·摩亞\n-\n劉曉彤\n佚名\n凱瑞·貝里·布洛甘\n-\n-\n\n\n茅十八\n馬宗德\n秦煌\n劉林\n秦煌\n-\n趙小銳\n晉松\n謝寧\n\n\n韋春花\n-\n白茵\n-\n羅冠蘭\n吳浣儀\n馬羚\n王琳\n黃小蕾\n\n\n李力世\n-\n談泉慶\n-\n-\n-\n-\n-\n-\n\n\n關安基\n-\n關鍵\n-\n-\n-\n-

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


retriever = vector_store.as_retriever(search_kwargs={"k": 5})

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)
query = "誰是蘇菲亞公主"
chain.invoke({"input": query})

In [40]:
query = "誰是蘇菲亞公主"
# Step 2: 把向量資料庫轉換為 retriever，並指定檢索參數
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

from langchain.retrievers import ContextualCompressionRetriever
rerank_retriever = ContextualCompressionRetriever(
    base_compressor=rerank, base_retriever=retriever
)

rerank_docs = rerank_retriever.invoke(query)
print(rerank_docs)
#pretty_print_docs(rerank_docs)

[Document(metadata={'source': 'https://zh.wikipedia.org/zh-tw/%E9%B9%BF%E9%BC%8E%E8%AE%B0', 'start_index': 2512, 'relevance_score': 0.5375269651412964}, page_content='人物介紹[編輯]\n主條目：鹿鼎記角色列表\n虛構人物[編輯]\n\n\n\n\n主角：\n\n韋小寶\n七名夫人：\n\n建寧公主\n沐劍屏\n方怡\n雙兒\n蘇荃\n曾柔\n阿珂\n\n\n\n\n宮中：\n\n海大富\n多隆\n張康年\n趙齊賢\n蕊初\n天地會青木堂：\n\n李力世\n關安基\n祁彪清\n玄貞道人\n徐天川\n風際中\n\n\n\n\n沐王府：\n\n方怡\n沐劍聲\n柳大洪\n吳立身\n劉一舟\n神龍教：\n\n教主洪安通\n假皇太后毛東珠\n瘦頭陀\n胖頭陀\n陸高軒\n\n\n\n\n其他：\n\n茅十八\n韋春花\n楊溢之\n李西華\n陶紅英\n莊三少奶\n晦聰\n澄觀\n阿琪\n胡逸之\n\n\n\n\n《碧血劍》：\n\n九難（長平公主）(《碧血劍》中的阿九)\n馮難敵\n歸辛樹\n歸二娘\n歸鍾\n何惕守\n\n\n\n\n歷史及傳說人物[編輯]\n\n\n\n\n朝廷：\n\n康熙（「小玄子」）\n孝莊文皇后\n鰲拜\n康親王\n索額圖\n行癡（順治帝）\n皇太后\n孝康章皇后\n建寧公主\n\n\n\n\n天地會：\n\n陳近南\n吳六奇\n台灣延平王府：\n\n鄭克塽\n馮錫範\n\n\n\n\n文士：\n\n呂留良\n黃宗羲\n顧炎武\n查繼佐\n明史案：\n\n莊允誠\n莊廷鑨\n吳之榮\n\n\n\n\n其他：\n\n葛爾丹\n桑結\n吳三桂\n吳應熊\n陳圓圓\n李自成\n\n\n\n\n施琅\n蘇菲亞公主\n瓦西里·瓦西里耶維奇·戈里津\n費岳多\n趙良棟\n張勇\n王進寶\n孫思克\n\n\n《碧血劍》人物[編輯]\n《鹿鼎記》一些人物，原先已在金庸另一部作品《碧血劍》出現，如九難、馮難敵父子、陳圓圓、李自成、歸辛樹一家、何惕守等。\n\n借用歷史[編輯]\n《鹿鼎記》的故事發生在清初，牽涉清朝朝廷與宮室。由於《鹿鼎記》是小說而非史實，當中不少情節所觸及的歷史事件，都在金庸的大膽創作

In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


retriever = vector_store.as_retriever(search_kwargs={"k": 1})

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(rerank_retriever, question_answer_chain)
query = "誰是蘇菲亞公主"
chain.invoke({"input": query})

{'input': '誰是蘇菲亞公主',
 'context': [Document(metadata={'source': 'https://zh.wikipedia.org/zh-tw/%E9%B9%BF%E9%BC%8E%E8%AE%B0', 'start_index': 2512, 'relevance_score': 0.5375269651412964}, page_content='人物介紹[編輯]\n主條目：鹿鼎記角色列表\n虛構人物[編輯]\n\n\n\n\n主角：\n\n韋小寶\n七名夫人：\n\n建寧公主\n沐劍屏\n方怡\n雙兒\n蘇荃\n曾柔\n阿珂\n\n\n\n\n宮中：\n\n海大富\n多隆\n張康年\n趙齊賢\n蕊初\n天地會青木堂：\n\n李力世\n關安基\n祁彪清\n玄貞道人\n徐天川\n風際中\n\n\n\n\n沐王府：\n\n方怡\n沐劍聲\n柳大洪\n吳立身\n劉一舟\n神龍教：\n\n教主洪安通\n假皇太后毛東珠\n瘦頭陀\n胖頭陀\n陸高軒\n\n\n\n\n其他：\n\n茅十八\n韋春花\n楊溢之\n李西華\n陶紅英\n莊三少奶\n晦聰\n澄觀\n阿琪\n胡逸之\n\n\n\n\n《碧血劍》：\n\n九難（長平公主）(《碧血劍》中的阿九)\n馮難敵\n歸辛樹\n歸二娘\n歸鍾\n何惕守\n\n\n\n\n歷史及傳說人物[編輯]\n\n\n\n\n朝廷：\n\n康熙（「小玄子」）\n孝莊文皇后\n鰲拜\n康親王\n索額圖\n行癡（順治帝）\n皇太后\n孝康章皇后\n建寧公主\n\n\n\n\n天地會：\n\n陳近南\n吳六奇\n台灣延平王府：\n\n鄭克塽\n馮錫範\n\n\n\n\n文士：\n\n呂留良\n黃宗羲\n顧炎武\n查繼佐\n明史案：\n\n莊允誠\n莊廷鑨\n吳之榮\n\n\n\n\n其他：\n\n葛爾丹\n桑結\n吳三桂\n吳應熊\n陳圓圓\n李自成\n\n\n\n\n施琅\n蘇菲亞公主\n瓦西里·瓦西里耶維奇·戈里津\n費岳多\n趙良棟\n張勇\n王進寶\n孫思克\n\n\n《碧血劍》人物[編輯]\n《鹿鼎記》一些人物，原先已在金庸另一部作品《碧血劍》出現，如九難、馮難敵父子、陳圓圓、李自成、歸辛樹一家、何惕守等。\n\n借用歷史[編輯]\n《鹿鼎記》的故事發生在清初，牽涉清朝朝廷與宮室。由於《鹿鼎記

### 📘 回家作業：RAG（Retrieval-Augmented Generation）

#### 📝 作業一：核心概念理解（30分）

請簡要回答下列問題

1. 請說明什麼是 RAG（Retrieval-Augmented Generation）？它與傳統的生成式語言模型有什麼不同？
2. 在 RAG 架構中，`Retriever` 和 `Generator` 各自的角色是什麼？它們如何互相配合？
3. 請說明 Embedding 的作用，以及為何要對文本進行向量化？

---

#### 🧑‍💻 作業二：實作練習（30分）

請根據課堂提供的 notebook《08-rag.ipynb》完成以下項目：

1. 嘗試將預設的資料集替換為你自己提供的任意一篇文章（可以是新聞、報導、科普等）。
2. 重新跑過向量嵌入（embedding）、建立向量庫（Vector Store），並透過 RAG Pipeline 提出三個問題，觀察答案是否合理。
3. 將你的文章及三個問答結果，整理為一個簡單的 Markdown 報告提交（附上截圖或輸出文字）。

---

#### 💡 作業三：應用場景發想（40分）

請構思一個你認為適合使用 RAG 技術的應用情境，並回答以下問題：

1. 你的應用要解決什麼問題？目標使用者是誰？
2. 為什麼使用 RAG 比純 LLM 或資料庫查詢更有優勢？
3. 在此情境中，你可能會蒐集什麼樣的資料做為知識庫？
4. 比較不同LLM模型在此議題的能力比較


